## Determining the optimal number of hidden layers and neurons for and Artificial Neural Network

Methods that can help with ANN

-Start Simple: Begin with a simple architecture and gradually increase complexity if needed.

-Grid Search/Random Search: Use grid search or random search to try different architectures.

-Cross-Validation: Use cross-validation to evaluate the performance of different architectures.

-Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
1. The number of neurons in the hidden layer should be between the size of the input layer and output later
2. A common practice is to start with 1-2 hidden layers.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
data=pd.read_csv('Churn_Modelling.csv')

In [5]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [7]:
one_hot_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=one_hot_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_geo.get_feature_names_out(['Geography']))

In [8]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [9]:
X=data.drop('Exited',axis=1)
y=data['Exited']

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [11]:
import pickle
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_geo.pkl','wb') as file:
    pickle.dump(one_hot_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [ ]:
#Define a function to create the model and try different parameters (Keras Classifier)
def create_model(neurons=32,layers=1): #a function to change the number of neurons and number of hidden layers using hyperparameter tuning
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],))) #

    for _ in range(layers-1): #adding extra hidden layers
        #if layers=1, then loop runs 0 times
        #if layers=3, then loop adds 2 more hidden layers
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid')) #output layer, binary classification and output between 0 and 1
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [15]:
#Create a keras classifier
model=KerasClassifier(build_fn=create_model,epochs=50,batch_size=10,verbose=0)